<a href="https://colab.research.google.com/github/Adhi1755/Pharma-Sales-Analysis/blob/main/Pharma_sales_data_analysis_and_forecasting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Pharma sales data analysis and forecasting**


## **Introduction**

The dataset is built from the initial dataset consisted of 600000 transactional data collected in 6 years (period 2014-2019), indicating date and time of sale, pharmaceutical drug brand name and sold quantity, exported from Point-of-Sale system in the individual pharmacy. Selected group of drugs from the dataset (57 drugs) is classified to the following Anatomical Therapeutic Chemical (ATC) Classification System categories:


1.   **M01AB** - Anti-inflammatory and antirheumatic products, non-steroids, Acetic acid derivatives and related substances
2.   **M01AE** - Anti-inflammatory and antirheumatic products, non-steroids, Propionic acid derivatives
3.   **M01AE** - Anti-inflammatory and antirheumatic products, non-steroids, Propionic acid derivatives
4.   **N02BA** - Other analgesics and antipyretics, Salicylic acid and derivatives
5.   **N02BE/B** - Other analgesics and antipyretics, Pyrazolones and Anilides
6.   **N05B** - Psycholeptics drugs, Anxiolytic drugs
7.   **N05B** - Psycholeptics drugs, Anxiolytic drugs
8.   **N05C** - Psycholeptics drugs, Hypnotics and sedatives drugs
9.   **R03** - Drugs for obstructive airway diseases
10.   **R06** - Antihistamines for systemic use

Sales data are resampled to the hourly, daily, weekly and monthly periods. Data is already pre-processed, where processing included outlier detection and treatment and missing data imputation.

## **Methodology**

The methodology for implementing this case study follows the typical time series forecasting pipeline, consisting of three major phases:

1.   Feature engineering and data preparation
2.   Exploratory data analysis (time-series analysis)
3.   Forecasting.










### **Feature engineering and data preparation**

Enabling the drive access

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


**Improting the required Libraries**

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

Dataset details:


1.   SalesDaily - contains daily sales data
2.   salesHourly - contains Hourly sales data
3.   salesMonthly - contains Monthly sales data
4.   salesWeekly - contains Weekly sales data






In [4]:
salesDaily = pd.read_csv('/content/drive/MyDrive/Google Colab/Pharma_Sales/salesdaily.csv')
salesHourly = pd.read_csv('/content/drive/MyDrive/Google Colab/Pharma_Sales/saleshourly.csv')
salesMonthly = pd.read_csv('/content/drive/MyDrive/Google Colab/Pharma_Sales/salesmonthly.csv')
salesWeekly = pd.read_csv('/content/drive/MyDrive/Google Colab/Pharma_Sales/salesweekly.csv')

In [5]:
salesDaily.head()

,datum,M01AB,M01AE,N02BA,N02BE,N05B,N05C,R03,R06,Year,Month,Hour,Weekday Name
0,1/2/2014,0.0,3.67,3.4,32.40,7.0,0.0,0.0,2.0,2014,1,248,Thursday
1,1/3/2014,8.0,4.00,4.4,50.60,16.0,0.0,20.0,4.0,2014,1,276,Friday
2,1/4/2014,2.0,1.00,6.5,61.85,10.0,0.0,9.0,1.0,2014,1,276,Saturday
3,1/5/2014,4.0,3.00,7.0,41.10,8.0,0.0,3.0,0.0,2014,1,276,Sunday
4,1/6/2014,5.0,1.00,4.5,21.70,16.0,2.0,6.0,2.0,2014,1,276,Monday


In [6]:
salesDaily.shape

(2106, 13)

In [7]:
salesDaily.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2106 entries, 0 to 2105
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   datum         2106 non-null   object 
 1   M01AB         2106 non-null   float64
 2   M01AE         2106 non-null   float64
 3   N02BA         2106 non-null   float64
 4   N02BE         2106 non-null   float64
 5   N05B          2106 non-null   float64
 6   N05C          2106 non-null   float64
 7   R03           2106 non-null   float64
 8   R06           2106 non-null   float64
 9   Year          2106 non-null   int64  
 10  Month         2106 non-null   int64  
 11  Hour          2106 non-null   int64  
 12  Weekday Name  2106 non-null   object 
dtypes: float64(8), int64(3), object(2)
memory usage: 214.0+ KB


As the date-time column is stored as an object, it is converted to the ***datetime*** format to support time-based indexing, resampling, and temporal analysis.




In [11]:
salesDaily['datum'] = pd.to_datetime(salesDaily['datum'])
display(salesDaily.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2106 entries, 0 to 2105
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   datum         2106 non-null   datetime64[ns]
 1   M01AB         2106 non-null   float64       
 2   M01AE         2106 non-null   float64       
 3   N02BA         2106 non-null   float64       
 4   N02BE         2106 non-null   float64       
 5   N05B          2106 non-null   float64       
 6   N05C          2106 non-null   float64       
 7   R03           2106 non-null   float64       
 8   R06           2106 non-null   float64       
 9   Year          2106 non-null   int64         
 10  Month         2106 non-null   int64         
 11  Hour          2106 non-null   int64         
 12  Weekday Name  2106 non-null   object        
dtypes: datetime64[ns](1), float64(8), int64(3), object(1)
memory usage: 214.0+ KB


None

In [8]:
salesDaily.describe()

,M01AB,M01AE,N02BA,N02BE,N05B,N05C,R03,R06,Year,Month,Hour
count,2106.000000,2106.000000,2106.000000,2106.000000,2106.000000,2106.000000,2106.000000,2106.000000,2106.000000,2106.000000,2106.000000
mean,5.033683,3.895830,3.880441,29.917095,8.853627,0.593522,5.512262,2.900198,2016.401235,6.344255,275.945869
std,2.737579,2.133337,2.384010,15.590966,5.605605,1.092988,6.428736,2.415816,1.665060,3.386954,1.970547
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2014.000000,1.000000,190.000000
25%,3.000000,2.340000,2.000000,19.000000,5.000000,0.000000,1.000000,1.000000,2015.000000,3.000000,276.000000
50%,4.990000,3.670000,3.500000,26.900000,8.000000,0.000000,4.000000,2.000000,2016.000000,6.000000,276.000000
75%,6.670000,5.138000,5.200000,38.300000,12.000000,1.000000,8.000000,4.000000,2018.000000,9.000000,276.000000
max,17.340000,14.463000,16.000000,161.000000,54.833333,9.000000,45.000000,15.000000,2019.000000,12.000000,276.000000


In [13]:
salesDaily.isna().sum()

,0
datum,0
M01AB,0
M01AE,0
N02BA,0
N02BE,0
N05B,0
N05C,0
R03,0
R06,0
Year,0


In [14]:
salesDaily.duplicated().sum()

np.int64(0)



The dataset contains no duplicate or missing values, indicating high data quality. With 2106 rows and 13 well-defined columns, the data types are correctly structured, making it ideal for EDA and time series forecasting tasks.



### **Exploratory data analysis (time-series analysis)**